
## 참조자료 

https://machinelearningmastery.com/keras-functional-api-deep-learning/

In [1]:
import keras 

Using TensorFlow backend.


# 1.  Keras models 

## Sequential model
-  Linear stack of layers
-  Useful for building simple models
- Simple classification network
- Encoder – Decoder models

In [2]:
from keras.models import Sequential
from keras.layers import Dense

##  Functional Model

- Multi – input and Multi –output models
- Complex models which forks into 2 or more branches
- Models with shared (Weights)layers

In [3]:
from keras.layers import Input
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, LSTM,  TimeDistributed
from keras.models import Model


# 2. Multi-Layer Perceptron

## 함수 기법으로 레이어 구성하기

In [ ]:
visible = Input(shape=(10,))
hidden1 = Dense(10, activation='relu')(visible)
hidden2 = Dense(20, activation='relu')(hidden1)
hidden3 = Dense(10, activation='relu')(hidden2)
output = Dense(1, activation='sigmoid')(hidden3)

## 모델 클래스로 모델 객체 만들기

In [4]:
model = Model(inputs=visible, outputs=output)
# summarize layers
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 20)                220       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 551
Trainable params: 551
Non-trainable params: 0
_________________________________________________________________
None


# 3.  Convolutional Neural Network

In [5]:
visible = Input(shape=(64,64,1))
conv1 = Conv2D(32, kernel_size=4, activation='relu')(visible)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = Conv2D(16, kernel_size=4, activation='relu')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)


flat = Flatten()(pool2)
hidden1 = Dense(10, activation='relu')(flat)
output = Dense(1, activation='sigmoid')(hidden1)


model = Model(inputs=visible, outputs=output)
# summarize layers
print(model.summary())

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64, 64, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 32)        544       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 27, 16)        8208      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2704)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                2705

# 4.  Recurrent Neural Network

In [6]:

visible = Input(shape=(100,1))
hidden1 = LSTM(10)(visible)
hidden2 = Dense(10, activation='relu')(hidden1)
output = Dense(1, activation='sigmoid')(hidden2)

model = Model(inputs=visible, outputs=output)


# summarize layers
print(model.summary())

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 100, 1)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 10)                480       
_________________________________________________________________
dense_7 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 11        
Total params: 601
Trainable params: 601
Non-trainable params: 0
_________________________________________________________________
None


# 5. Shared Layers Model

In [7]:
from keras.layers import concatenate

### Shared Input Layer

하나의 입력 계층을 가지고 두 개의 레이어 계층으로 별도의 처리를 한 후에 하나로 병합해서 결과를 처리한다. 

In [8]:
# input layer
visible = Input(shape=(64,64,1))

# first feature extractor
conv1 = Conv2D(32, kernel_size=4, activation='relu')(visible)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
flat1 = Flatten()(pool1)

# second feature extractor
conv2 = Conv2D(16, kernel_size=8, activation='relu')(visible)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
flat2 = Flatten()(pool2)

# merge feature extractors
merge = concatenate([flat1, flat2])

# interpretation layer
hidden1 = Dense(10, activation='relu')(merge)

# prediction output
output = Dense(1, activation='sigmoid')(hidden1)

model = Model(inputs=visible, outputs=output)
# summarize layers
print(model.summary())

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 61, 61, 32)   544         input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 57, 57, 16)   1040        input_4[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_3 (MaxPooling2D)  (None, 30, 30, 32)   0           conv2d_3[0][0]                   
____________________________________________________________________________________________

### Shared Feature Extraction Layer

두 개의 레이어 계층을 분리해서 하나는 하나의 계층만 통과시키고 나머지는 세개의 계층을 통과시킨 후에 병합해서 결과를 처리한다.


In [9]:
# define input
visible = Input(shape=(100,1))

# feature extraction
extract1 = LSTM(10)(visible)

# first interpretation model
interp1 = Dense(10, activation='relu')(extract1)

# second interpretation model
interp11 = Dense(10, activation='relu')(extract1)
interp12 = Dense(20, activation='relu')(interp11)
interp13 = Dense(10, activation='relu')(interp12)

# merge interpretation
merge = concatenate([interp1, interp13])

# output
output = Dense(1, activation='sigmoid')(merge)

model = Model(inputs=visible, outputs=output)

# summarize layers
print(model.summary())

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 100, 1)       0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 10)           480         input_5[0][0]                    
__________________________________________________________________________________________________
dense_12 (Dense)                (None, 10)           110         lstm_2[0][0]                     
__________________________________________________________________________________________________
dense_13 (Dense)                (None, 20)           220         dense_12[0][0]                   
____________________________________________________________________________________________

## Multiple Input Model

두 개의 입력계층을 별도로 처리한 후에 이를 병합해서 하나의 결과로 처리한다

In [10]:
# first input model
visible1 = Input(shape=(64,64,1))
conv11 = Conv2D(32, kernel_size=4, activation='relu')(visible1)
pool11 = MaxPooling2D(pool_size=(2, 2))(conv11)
conv12 = Conv2D(16, kernel_size=4, activation='relu')(pool11)
pool12 = MaxPooling2D(pool_size=(2, 2))(conv12)
flat1 = Flatten()(pool12)

# second input model
visible2 = Input(shape=(32,32,3))
conv21 = Conv2D(32, kernel_size=4, activation='relu')(visible2)
pool21 = MaxPooling2D(pool_size=(2, 2))(conv21)
conv22 = Conv2D(16, kernel_size=4, activation='relu')(pool21)
pool22 = MaxPooling2D(pool_size=(2, 2))(conv22)
flat2 = Flatten()(pool22)

# merge input models
merge = concatenate([flat1, flat2])

# interpretation model
hidden1 = Dense(10, activation='relu')(merge)
hidden2 = Dense(10, activation='relu')(hidden1)
output = Dense(1, activation='sigmoid')(hidden2)

model = Model(inputs=[visible1, visible2], outputs=output)

# summarize layers
print(model.summary())

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 61, 61, 32)   544         input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 29, 29, 32)   1568        input_7[0][0]                    
____________________________________________________________________________________________

## Multiple Output Model

In [11]:
# input layer
visible = Input(shape=(100,1))

# feature extraction
extract = LSTM(10, return_sequences=True)(visible)

# classification output
class11 = LSTM(10)(extract)
class12 = Dense(10, activation='relu')(class11)
output1 = Dense(1, activation='sigmoid')(class12)

# sequence output
output2 = TimeDistributed(Dense(1, activation='linear'))(extract)

# output
model = Model(inputs=visible, outputs=[output1, output2])

# summarize layers
print(model.summary())

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 100, 1)       0                                            
__________________________________________________________________________________________________
lstm_3 (LSTM)                   (None, 100, 10)      480         input_8[0][0]                    
__________________________________________________________________________________________________
lstm_4 (LSTM)                   (None, 10)           840         lstm_3[0][0]                     
__________________________________________________________________________________________________
dense_19 (Dense)                (None, 10)           110         lstm_4[0][0]                     
____________________________________________________________________________________________

In [12]:
inputs = Input(shape=(7,6)) # 7 steps ahead and 6 variables

m = Dense(64,activation='relu')(inputs)
m = Dense(64,activation='relu')(m)

# 아웃 게층 분리
outputA = Dense(1,activation='tanh')(m)
outputB = Dense(1,activation='tanh')(m)
 
model = Model(inputs=[inputs], outputs=[outputA, outputB])


print(model.summary())

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 7, 6)         0                                            
__________________________________________________________________________________________________
dense_22 (Dense)                (None, 7, 64)        448         input_9[0][0]                    
__________________________________________________________________________________________________
dense_23 (Dense)                (None, 7, 64)        4160        dense_22[0][0]                   
__________________________________________________________________________________________________
dense_24 (Dense)                (None, 7, 1)         65          dense_23[0][0]                   
____________________________________________________________________________________________